In [1]:
# 

# Enable autoreloading if import packages are changed
%load_ext autoreload
%autoreload 2

# Set up python path 
import sys
import os
import cv2
import numpy as np 



import torch 
import matplotlib.pyplot as plt
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu") #we assume the default setting will be cpu for better testing 

In [4]:
import src.match as matcher
from skimage.feature import plot_matches
import os

SIFT  = 0
ORB   = 1
BRIEF = 2 
KEYPOINT_NUM_THREHOLD = 10
NEED_COLOR = False
isDebug =  False #also read out the file to plot the correspondences, just for two chess imgs

def extract_features(imgs, method_name = SIFT):
    """
    output:
    keypointxy_for_all: shape(list) = [images, num_of_keypoint, x, y]
    descriptor_foa_all: shape(list) = [images, num_of_keypoint, descriptor]
    colors_for_all: shape(list) = [images, num_of_keypoint, color]
    """
    if method_name == SIFT: extractor = cv2.SIFT_create() #default (int nfeature=0, int nOctaveLayers=3, double contrastThreshold = 0.04)
    elif method_name == ORB: extractor = cv2.ORB_create()
    elif method_name == BRIEF: 
        fast = cv2.FastFeatureDetector_create() 
        brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()
    keypointxy_for_all = []
    descriptor_for_all = []
    colors_for_all = [] #for later shading, default is no use of this
    for img in imgs:

        if img is None:
            continue
        if method_name != SIFT and method_name!= ORB:
            keypoints = fast.detect(img, None)
            keypoints, descriptors = brief.compute(img,keypoints)
        else:    
            keypoints, descriptors = extractor.detectAndCompute(cv2.cvtColor(img,cv2.COLOR_BGR2GRAY),None)
        
        if len(keypoints) <= KEYPOINT_NUM_THREHOLD: continue
            
        # print(descriptors.shape)

        pt = np.array([kp.pt for kp in keypoints])
        # print(pt.shape)
        keypointxy_for_all.append(pt)
        descriptor_for_all.append(descriptors)

        if NEED_COLOR:
            colors = np.zeros((len(keypoints),3))
            for i, kp in enumerate(keypoints):
                p = kp.pt
                colors[i] = img[int(p[1])][int(p[0])]
            colors_for_all.append(colors)
    
    # for_all are all list instead of numpy array, due to diff dimensions of diff imgs
    if NEED_COLOR:
        return np.array(keypointxy_for_all), np.array(descriptor_for_all), np.array(colors_for_all)
    else:
        return keypointxy_for_all, descriptor_for_all
    
def feature_matching_file(keypointxy_for_all, descriptor_for_all, method_name, obj_name, num_of_match_threshold=10, match_score_threshold=70):
    """
    Function: iterate over all images and generate the matching file 
    NOTE: might need to fine-tune the threshold to generate both sufficient and good matches
    
    Outputs:
    method_name:             SIFT, BRIEF, SURF, etc.
    num_of_match_threshold:  if num_of_match < this, skip generating correspondence for these two images
    match_score_threshold:   if match_score < this, skip the match
    """

    for i in range(len(descriptor_for_all)):
        for j in range(i + 1, len(descriptor_for_all)):
            matches_for_one_img = []    

            try:
                matches_idx = matcher.match(
                    descriptors1=torch.tensor(descriptor_for_all[i], device=device, dtype=torch.float32),
                    descriptors2=torch.tensor(descriptor_for_all[j], device=device, dtype=torch.float32),
                    device="cpu",
                    dist='hamming',
                    ratio=0.99,
                    threshold=match_score_threshold
                ).detach().cpu().numpy()
            except Exception as e:
                print(f"Error in matching descriptors for image {i} and image {j}: {e}")
                continue

            if len(matches_idx) < num_of_match_threshold:
                continue

            print(f'img{i} has {len(matches_idx)} matches with img{j}')

            for k in range(len(matches_idx)):
                single_match = np.array([keypointxy_for_all[i][matches_idx[k][0]], keypointxy_for_all[j][matches_idx[k][1]]]).flatten()
                # assert single_match.shape==(4,)
                matches_for_one_img.append(single_match)

            try:
                np.savetxt(f"./corres/{obj_name}/{method_name}_corres/{i}_{j}.txt", matches_for_one_img)
                print(f'./corres/{obj_name}/{method_name}_corres/{i}_{j}.txt is generated')
            except Exception as e:
                print(f"Error in saving correspondence file for image {i} and image {j}: {e}")
        
        break

def generate_SIFT_corres(imgs, obj_name, num_of_match_threshold=10, match_threhold=70):
    """
    details abt two threshold are in feature_matching_file function
    """
    kps, descs = extract_features(imgs, SIFT)
    feature_matching_file(kps,descs,'SIFT',obj_name, num_of_match_threshold, match_threhold)

def generate_BRIEF_corres(imgs, obj_name, num_of_match_threshold=10, match_threhold=10):
    """
    details abt two threshold are in feature_matching_file function
    """
    kps, descs = extract_features(imgs, BRIEF)
    feature_matching_file(kps,descs,'BRIEF',obj_name, num_of_match_threshold, match_threhold)

def generate_ORB_corres(imgs, obj_name, num_of_match_threshold=10, match_threhold=20):
    """
    details abt two threshold are in feature_matching_file function
    """
    kps, descs = extract_features(imgs, ORB)
    feature_matching_file(kps,descs,'ORB',obj_name, num_of_match_threshold, match_threhold)


target_object = 'milk' #NOTE: change this to generate matches for different objects
match_threshold_SIFT = 70
match_threshold_BRIEF = 10
match_threshold_ORB = 20
num_of_match_threshold = 10

path = "C:/Users/mrina/OneDrive/Desktop/3dproj/stage2_v2/stage2/data/{}/images/".format(target_object) #NOTE: please change the path or put the image files accordingly
files = os.listdir(path)
# print(files)
imgs = []
for file in files:
    if not os.path.isdir(file):
        # print(path+"/"+file)
        img = cv2.imread(path+"/"+file);
        imgs.append(img)
if isDebug: assert len(imgs)==2
print(np.array(imgs).shape)


generate_SIFT_corres(imgs, target_object,num_of_match_threshold, match_threshold_SIFT)
# generate_ORB_corres(imgs, target_object,num_of_match_threshold, match_threshold_ORB)
# generate_BRIEF_corres(imgs, target_object,num_of_match_threshold, match_threshold_BRIEF)




(50, 1080, 1920, 3)
tensor([])
tensor([], dtype=torch.int64)
tensor([])
tensor([], dtype=torch.int64)
tensor([])
tensor([], dtype=torch.int64)
tensor([])
tensor([], dtype=torch.int64)
tensor([])
tensor([], dtype=torch.int64)
tensor([])
tensor([], dtype=torch.int64)
tensor([])
tensor([], dtype=torch.int64)
tensor([])
tensor([], dtype=torch.int64)
tensor([])
tensor([], dtype=torch.int64)
tensor([])
tensor([], dtype=torch.int64)
tensor([])
tensor([], dtype=torch.int64)
tensor([])
tensor([], dtype=torch.int64)
tensor([])
tensor([], dtype=torch.int64)
tensor([])
tensor([], dtype=torch.int64)
tensor([])
tensor([], dtype=torch.int64)
tensor([])
tensor([], dtype=torch.int64)
tensor([])
tensor([], dtype=torch.int64)
tensor([])
tensor([], dtype=torch.int64)
tensor([])
tensor([], dtype=torch.int64)
tensor([])
tensor([], dtype=torch.int64)
tensor([])
tensor([], dtype=torch.int64)
tensor([])
tensor([], dtype=torch.int64)
tensor([])
tensor([], dtype=torch.int64)
tensor([])
tensor([], dtype=torch.int

KeyboardInterrupt: 

### only for combined matches

In [13]:

def feature_matching_file_combined(keypointxy_for_all, descriptor_for_all,  obj_name, num_of_match_threshold = 16, match_score_threshold_SIFT =70, match_score_threshold_ORB=20, match_score_threshold_BRIEF=10):
    """
    function: iterate over all images and generate the matching file 
    NOTE: might need to finetune the threhold to generate both sufficient and good matches

    input:
    keypointxy_for_all: shape(list) = [3,images, num_of_keypoint, x, y]
    descriptor_foa_all: shape(list) = [3,images, num_of_keypoint, descriptor]

    outputs:
    method_name:            SIFT, BRIEF, SURF and etc.
    num_of_match_threshold: if num_of_match < this, skip generating correspondence for these two images
    match_score_threshold:  if match_score < this, skip the match
    """
    
    for i in range(len(descriptor_for_all)-1):        
        for j in range(i+1, len(descriptor_for_all)):
            

            matches_idx_SIFT = matcher.match(
                                        descriptors1 = torch.tensor(descriptor_for_all[SIFT][i], device = device,dtype = torch.float32),
                                        descriptors2 = torch.tensor(descriptor_for_all[SIFT][j], device = device,dtype = torch.float32),
                                        device="cpu",
                                        dist='hamming',
                                        ratio=0.95,
                                        threshold = match_score_threshold_SIFT,
                                        isdebug = False
                                    ).detach().cpu().numpy()
            matches_idx_ORB = matcher.match(
                                        descriptors1 = torch.tensor(descriptor_for_all[ORB][i], device = device,dtype = torch.float32),
                                        descriptors2 = torch.tensor(descriptor_for_all[ORB][j], device = device,dtype = torch.float32),
                                        device="cpu",
                                        dist='hamming',
                                        ratio=0.95,
                                        threshold = match_score_threshold_ORB,
                                        isdebug = False
                                    ).detach().cpu().numpy()
            matches_idx_BRIEF = matcher.match(
                                        descriptors1 = torch.tensor(descriptor_for_all[BRIEF][i], device = device,dtype = torch.float32),
                                        descriptors2 = torch.tensor(descriptor_for_all[BRIEF][j], device = device,dtype = torch.float32),
                                        device="cpu",
                                        dist='hamming',
                                        ratio=0.95,
                                        threshold = match_score_threshold_BRIEF,
                                        isdebug = False
                                    ).detach().cpu().numpy()
            
            num_of_match = len(matches_idx_ORB)+len(matches_idx_BRIEF)+len(matches_idx_SIFT)
            if(num_of_match < num_of_match_threshold): continue

            print('img{} has {} matches with img{}'.format(i,num_of_match,j))
            print('./corres/{}/combined_corres/{}_{}.txt is generated'.format(obj_name, i,j))

            matches_for_one_img = []
            matches_SIFT  = []
            matches_ORB   = []
            matches_BRIEF = []
            
            for k in range(get_max_matches_num(len(matches_idx_SIFT),len(matches_idx_BRIEF),len(matches_idx_ORB) )):

                single_match = np.array([keypointxy_for_all[i][matches_idx[k][0]], keypointxy_for_all[j][matches_idx[k][1]]]).flatten()
                
                assert single_match.shape==(4,)
                matches_for_one_img.append(single_match)

    return 0

def get_max_matches_num(x,y,z):
    if  x >=y and x>=z: return x
    elif y>=x and y>=z: return y 
    else:               return z 

def generate_combined_corres(imgs, obj_name, num_of_match_threshold = 10):
    """
    details abt two threshold are in feature_matching_file function
    """

    kps_SIFT,  descs_SIFT  = extract_features(imgs, SIFT)
    kps_ORB,   descs_ORB   = extract_features(imgs, ORB)
    kps_BRIEF, descs_BRIEF = extract_features(imgs, BRIEF)
    kps_combined   = [  kps_SIFT,   kps_ORB,   kps_BRIEF]
    descs_combined = [descs_SIFT, descs_ORB, descs_BRIEF]
    
    feature_matching_file_combined( kps_combined, descs_combined, obj_name, num_of_match_threshold, match_threshold_SIFT )


generate_combined_corres(imgs, target_object)





AttributeError: module 'cv2' has no attribute 'xfeatures2d'

In [ ]:
def splicing_methods(num_of_imgs):
    for i in range(num_of_imgs-1):
        for j in range(i+1,num_of_imgs):
            match1 = np.array(np.loadtxt('./'))
files = os.listdir(path)
splicing_methods('SIFT', 'BRIEF', 'ORB',len(files))

In [ ]:
# only for debugging
# checking if the plots are the same after reading from files, please correct the path to corresponding methods

if isDebug:
    match = np.array(np.loadtxt('./SIFT_corres/0_1.txt')) 
    print(match.shape)
    fig, axs = plt.subplots(figsize=(30.0, 20.0))
    k = 10
    idx = np.array([[i,i] for i in range(10)])
    plot_matches(
        axs,
        gray1,
        gray2,
        match[:,0:2],
        match[:,2:4],
        idx,
        alignment="horizontal",
        only_matches=True,
    )
    plt.show()

